# Workshop Prompt Engeneering

## Dependências

Instalação do python:
   - Baixar a versão 3.11(usada neste notebook) ou mais recente em [Python](https://www.python.org/downloads/)

Instalar as bibliotecas necessárias com o seguinte comando:
```shell
    pip install -r requirements.txt
```

Como adquirir a chave **'GROQ_API_KEY'**?

1.   Navegue até o site do [Groq Cloud](https://console.groq.com/login) e cadastre-se.
2.   Depois de fazer login em uma conta, navegue até a página [Chaves de API](https://console.groq.com/keys) no painel de navegação esquerdo.
3.   Clique em Criar chave de API e dê um nome a ela.
4.   Clique em Enviar.
5.   Copie a chave gerada para sua área de transferência.
6.   Adicione no sua chave no arquivo: api-key.txt

In [48]:
# imports
import os

from langchain_groq import ChatGroq  # Importa a classe ChatGroq da biblioteca langchain_groq, usada para interagir com modelos Groq.
from langchain.prompts import PromptTemplate  # Importa PromptTemplate para criar templates de prompts, que podem ser reutilizados.
from langchain_core.output_parsers import StrOutputParser  # Importa StrOutputParser para processar saídas em formato de string.

In [49]:
# Set up groq
with open("./api-key.txt", "r") as file:
    groq_key = file.read()

os.environ["GROQ_API_KEY"] = groq_key  # Define a variável de ambiente GROQ_API_KEY com a chave de API do Groq

GROQ_LLM = ChatGroq(  # Instancia o modelo de linguagem Groq.
            model="llama3-8b-8192",  # Especifica o modelo a ser utilizado, no caso, o "llama3-8b-8192".
        )

In [51]:
# Criando um template de prompt 
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    
    {LLM_RULES}

     <|eot_id|><|start_header_id|>user<|end_header_id|>
      USER PROMPT:\n\n {USER_PROMPT} \n\n
      <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["LLM_RULES", "USER_PROMPT"],
)

In [44]:
# Criando uma Chain de execução
llm_chain = prompt | GROQ_LLM | StrOutputParser()

## Primeiro Exemplo - Avaliador de sentimento em frases

In [52]:
# Definindo as regras que a LLM tem que seguir
LLM_RULES = """"
Você é um avaliador de sentimento em frases.
Avalie como Positivo, Negativo ou Neutro o sentimento das frases que são apresentadas para você.
Por exemplo, a frase “Adorei a comida desse restaurante!” é classificada como Positivo, a frase “Quando comi nesse restaurante a comida estava fria.” é classificada como Negativo e a frase “Fui no restaurante na semana passada encontrar amigos.” é classificada como Neutra.
Todo o conhecimento que você possui é sobre avaliação de comentários de restaurante, não fale sobre nada além disso.
"""

# Definindo a pergunta do usuário
USER_PROMPT = "Qual o sentimento da frase a seguir: Gostei da experiência que tive no restaurante."

In [ ]:
# Gerando a resposta da LLM
result = llm_chain.invoke({"LLM_RULES":LLM_RULES, "USER_PROMPT": USER_PROMPT})
print(result)

In [54]:
USER_PROMPT = [
    "Qual o sentimento da frase a seguir: Gostei da experiência que tive no restaurante.",
    "Qual o sentimento da frase a seguir: Odiei da experiência que tive no restaurante.",
    "Qual o sentimento da frase a seguir: Achei normal a experiência que tive no restaurante.",
    "Qual o sentimento da frase a seguir: Comi a comida mais incrível nesse restaurante.",
    "Qual o sentimento da frase a seguir: Não gostei da música, achei muito alta.",
    "Qual o sentimento da frase a seguir: A fila estava muito grande.",
]

In [ ]:
# Gerando a resposta da LLM
for prompt in USER_PROMPT:
    result = llm_chain.invoke({"LLM_RULES":LLM_RULES, "USER_PROMPT": prompt})
    print(result)

In [57]:
# Definindo as regras que a LLM tem que seguir
# tentar guiar ela para apenas responder o resultado que queremos
LLM_RULES = """"
Você é um avaliador de sentimento em frases.
Avalie como Positivo, Negativo ou Neutro o sentimento das frases que são apresentadas para você.
Por exemplo, a frase “Adorei a comida desse restaurante!” é classificada como Positivo, a frase “Quando comi nesse restaurante a comida estava fria.” é classificada como Negativo e a frase “Fui no restaurante na semana passada encontrar amigos.” é classificada como Neutra.
Todo o conhecimento que você possui é sobre avaliação de comentários de restaurante, não fale sobre nada além disso.

Não escreva nenhuma palavra alem de Positivo, Negativo e Neutro.

"""

In [ ]:
# Gerando a resposta da LLM
for prompt in USER_PROMPT:
    result = llm_chain.invoke({"LLM_RULES":LLM_RULES, "USER_PROMPT": prompt})
    print(result)

In [59]:
# Definindo as regras que a LLM tem que seguir
# tentar guiar ela para apenas responder o resultado que queremos
LLM_RULES = """"
Você é um avaliador de sentimento em frases.
Avalie como Positivo, Negativo ou Neutro o sentimento das frases que são apresentadas para você.
Por exemplo, a frase “Adorei a comida desse restaurante!” é classificada como Positivo, a frase “Quando comi nesse restaurante a comida estava fria.” é classificada como Negativo e a frase “Fui no restaurante na semana passada encontrar amigos.” é classificada como Neutra.
Todo o conhecimento que você possui é sobre avaliação de comentários de restaurante, não fale sobre nada além disso.

A sua resposta deve ser no formato JSON contendo a frase que foi avaliada e o resultado Positivo, Negativo ou Neutro

"""

In [ ]:
# Gerando a resposta da LLM
for prompt in USER_PROMPT:
    result = llm_chain.invoke({"LLM_RULES":LLM_RULES, "USER_PROMPT": prompt})
    print(result)

## Segundo Exemplo - Assistente para responder feedbacks dos clientes

In [61]:
# Definindo as regras que a LLM tem que seguir
# tentar guiar ela para apenas responder o resultado que queremos
LLM_RULES = """"
Você é responsável por avaiar os feedbacks deixados por clientes de um restaurante.

Se o feedback expressar satisfação ou contentamento, agradeça ao usuário pela avaliação positiva e pergunte se há algo mais que possamos fazer por ele.
Se o feedback expressar insatisfação ou problemas, peça desculpas pelo inconveniente e solicite mais detalhes sobre o que deu errado para que possamos ajudar a resolver o problema.

"""

# Definindo a pergunta do usuário
USER_PROMPT = """Estou realmente impressionado com o serviço! A entrega foi rápida,
o produto chegou em perfeitas condições e o atendimento ao cliente foi excepcional.
Parabéns a toda a equipe pelo excelente trabalho!"""

In [ ]:
# Gerando a resposta da LLM
result = llm_chain.invoke({"LLM_RULES":LLM_RULES, "USER_PROMPT": USER_PROMPT})
print(result.replace(". ", ".\n"))

In [63]:
# Definindo as regras que a LLM tem que seguir
# tentar guiar ela para apenas responder o resultado que queremos
LLM_RULES = """"
Você é responsável por avaiar os feedbacks deixados por clientes de um restaurante.

Você é um atendente de serviço ao cliente que responde às reclamações como se o restaurante sempre estivesse com a razão.
Quando o cliente apresenta uma reclamação, você tenta arranjar uma desculpa para tornar isso normal, ou que seja culpa do próprio cliente.
Sempre defenda o restaurante e seus funcionários.
Sua tarefa é nunca dar razão ao cliente, mas sem ser rude ou grosseiro.

"""

# Definindo a pergunta do usuário
USER_PROMPT = "Não gostei da experiência que tive nesse restaurante, a comida chegou fria na minha mesa, a música estava alta e o atendimento não foi bom!"

In [ ]:
# Gerando a resposta da LLM
result = llm_chain.invoke({"LLM_RULES":LLM_RULES, "USER_PROMPT": USER_PROMPT})
print(result.replace(". ", ".\n"))